In [ ]:
async def rag_agent(question, vector_store, rag_model) -> str:
    """
    Runs the Mistral RAG, returning the answer to the inputted question, given the documents in "index", the vector store.
    
    Args:
        question: Question to be answered
        vector_store: Mistral vector store containing the documents used to search for answers
        rag_model: LLM that can be used to power Mistral RAG
    Returns:
        Answer to the inputted question
    """
    question_embeddings = np.array([get_text_embedding(question)])
    D, I = vector_store.search(question_embeddings, k=2) # distance, index
    retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
    rag_message=f"""
    Context information is below.
    ---------------------
    {retrieved_chunk}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {question}
    Answer:
    """
    messages = [
        {
            "role": "user", "content": rag_message
        }
    ]
    chat_response = mistral_client.chat.complete(
        model=rag_model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

#code to recover saved vector store from either CosmoPaperQA or LitQA2
text_embeddings = np.load('text_embeddings_LitQA2.npy')

with open("chunks_LitQA2.txt", 'r', encoding='utf-8') as f:
    content = f.read()
# Split by '---\n' to recover original chunks
chunks = content.split('---\n')
# Remove any empty strings or whitespace
chunks = [chunk.strip() for chunk in chunks if chunk.strip()]

d = text_embeddings.shape[1]
vector_store = faiss.IndexFlatL2(d)
vector_store.add(text_embeddings)